## Identify patients' diagnosis before liver tranplation index date

In [8]:
in_path = '../Liver_optum/outputs/'

In [5]:
import pandas as pd
import numpy as np

# Filter diagnosis records before Index Day

In [35]:
diag = pd.read_csv(in_path+"final_diag_all.csv", sep = "\t")
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16016916 entries, 0 to 16016915
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   patid     int64 
 1   fst_dt    object
 2   icd_flag  int64 
 3   diag      object
dtypes: int64(2), object(2)
memory usage: 488.8+ MB


# Define inclusion timestamp as 3 year before index day

In [56]:
b4 = -365*3
b4

-1095

In [58]:
diag_idx['b4Idx3yr'] =np.where(diag_idx.day_gap.between(b4,-1),1,0)

Generate table df that contains only inclusion diagnosis event

In [ ]:
df = diag_idx.loc[diag_idx.b4Idx3yr==1].reset_index(drop=True)


## Label diagnosis table for patients

In [61]:
liver = ["Z4823","Z944","T8649","T8640","T8643","V427","99682","47133","47135","47140","47141","47142","47143","47144","47145","47146","47147","0FY00Z0","50.5"]


atrial_fibrillation = ["I480","I4891","I482","I4819","I4820","I481","I489","I48","I4811","I4821","I483","I484","I4892", "42731", "42732"]


### check if any patient did the liver transplant before index

In [ ]:
df["liver"] = df["diag"].apply(lambda x:1 if x in liver else 0)

In [67]:
df.liver.value_counts()

0    3869795
Name: liver, dtype: int64

In [71]:
del df['liver']
del df['b4Idx3yr']

In [73]:
df.to_csv(in_path+'temp_diag_3yr_b4index.csv', index=False)

In [75]:
Myocardial_Infarction_icd10 = ["I219","I21A9","I21A1","I21","I214","I210","I211","I212","I222","I213","I228","I22","I2111","I230","I2129","I221","I2121","I232","I234","I220","I2119","I2109","I231","I2101","I235","I236","I2102","I21A"]
Myocardial_Infarction_icd9 = ["41090","41091","41092","41071","41051","41061","41081","41031","42979","41021","41041","42971","4295","41001","41011","4296","41002","41010","41012","41020","41022","41030","41032","41040","41042","41050","41052","41060","41062","41070","41072","41080","41082"]
Myocardial_Infarction = Myocardial_Infarction_icd10 + Myocardial_Infarction_icd9


df["MI_b4Idx_oldCode"] = df["diag"].apply(lambda x:1 if x in Myocardial_Infarction else 0)

In [77]:
Myocardial_Infarction_index_day = df.loc[(df["MI_b4Idx_oldCode"] == 1)].groupby("patid").fst_dt.min().reset_index().rename(columns={'fst_dt':'MI_b4Idx_oldCode_index_day'}).drop_duplicates()


In [79]:
Myocardial_Infarction_index_day.to_csv(in_path+'temp_MI_b4Idx_oldCode_index_day.csv',index=False)

## Process with new ICD codes

In [91]:
import os

def patient(df, ls):
    # patient level distribution
    cl = ['patid',ls]
    total_pat = df.patid.nunique()
    check = df[cl].drop_duplicates().reset_index(drop=True)
    gb = pd.DataFrame(check.groupby(check[ls]).count().to_records())
    gb['rate'] = (gb['patid']/total_pat)*100
    return gb.style.format({'rate': '{0:.4}%'})

icd_path = '/ctao_shared_data/Liver_optum/dict_icd/'
dx_path = icd_path+'dx/'
os.listdir(dx_path)

['alcohol_9.pickle',
 'alcohol_10.pickle',
 'CAD_9.pickle',
 'CAD_10.pickle',
 'hypertension_9.pickle',
 'hypertension_10.pickle',
 't2dm_9.pickle',
 't2dm_10.pickle',
 'MI_9.pickle',
 'MI_10.pickle',
 'tobacco_9.pickle',
 'tobacco_10.pickle',
 'PVD_9.pickle',
 'PVD_10.pickle',
 'dyslipidemia_9.pickle',
 'dyslipidemia_10.pickle',
 'stroke_9.pickle',
 'phtn_9.pickle',
 'bmi_9.pickle',
 'bmi_10.pickle',
 'stroke_10.pickle',
 'phtn_10.pickle']

### Alcohol before index

In [88]:
al_9 = pd.read_pickle(dx_path+'alcohol_9.pickle')
al_10 = pd.read_pickle(dx_path+'alcohol_10.pickle')
al_ls = al_9 + al_10
len(al_9), len(al_10),len(al_ls), al_ls [:3]

(10, 62, 72, ['30502', '30500', '30390'])

In [92]:
df['alcohol_b4idx'] = np.where(df.diag.isin(al_ls),1,0)
patient(df, 'alcohol_b4idx')

,alcohol_b4idx,patid,rate
0,0,16558,99.98%
1,1,1941,11.72%


### CAD

In [95]:
fn = 'CAD'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(14, 16, 30, ['30502', '30500', '30390'])

In [96]:
df['CAD_b4idx'] = np.where(df.diag.isin(all_ls),1,0)
patient(df, 'CAD_b4idx')

,CAD_b4idx,patid,rate
0,0,16559,99.98%
1,1,2944,17.78%


### Hypertension

In [97]:
fn = 'hypertension'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(50, 1, 51, ['30502', '30500', '30390'])

In [98]:
df['hypertension_b4idx'] = np.where(df.diag.isin(all_ls),1,0)
patient(df, 'hypertension_b4idx')

,hypertension_b4idx,patid,rate
0,0,16549,99.92%
1,1,9105,54.98%


### Diabete

In [99]:
fn = 't2dm'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(30, 116, 146, ['30502', '30500', '30390'])

In [100]:
df['diabeteMCIcode_b4idx'] = np.where(df.diag.isin(all_ls),1,0)
patient(df, 'diabeteMCIcode_b4idx')

,diabeteMCIcode_b4idx,patid,rate
0,0,16525,99.78%
1,1,6002,36.24%


### MI 

In [101]:
fn = 'MI'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(1, 1, 2, ['30502', '30500', '30390'])

In [102]:
df['MCI_sl_b4idx'] = np.where(df.diag.isin(all_ls),1,0) # MCI short list
patient(df, 'MCI_sl_b4idx')

,MCI_sl_b4idx,patid,rate
0,0,16562,100.0%
1,1,593,3.58%


In [103]:
## Compare to MI codes for after index identification

patient(df, 'MI_b4Idx_oldCode')

,MI_b4Idx_oldCode,patid,rate
0,0,16562,100.0%
1,1,511,3.085%


### Tobacco

In [104]:
fn = 'tobacco'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(2, 12, 14, ['30502', '30500', '30390'])

In [105]:
df['tobacco_b4idx'] = np.where(df.diag.isin(all_ls),1,0) 
patient(df, 'tobacco_b4idx')

,tobacco_b4idx,patid,rate
0,0,16561,99.99%
1,1,3669,22.15%


### PVD 

In [106]:
fn = 'PVD'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(1, 1, 2, ['30502', '30500', '30390'])

In [107]:
df['PVD_b4idx'] = np.where(df.diag.isin(all_ls),1,0) 
patient(df, 'PVD_b4idx')

,PVD_b4idx,patid,rate
0,0,16561,99.99%
1,1,953,5.754%


### dyslipidemia

In [108]:
fn = 'dyslipidemia'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(1, 1, 2, ['30502', '30500', '30390'])

In [109]:
df['dyslipidemia_b4idx'] = np.where(df.diag.isin(all_ls),1,0) 
patient(df, 'dyslipidemia_b4idx')

,dyslipidemia_b4idx,patid,rate
0,0,16561,99.99%
1,1,4995,30.16%


### Stroke 

In [110]:
fn = 'stroke'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(1, 1, 2, ['30502', '30500', '30390'])

In [115]:
df['stroke_b4idx'] = np.where(df.diag.isin(all_ls),1,0) 
patient(df, 'stroke_b4idx')

,stroke_b4idx,patid,rate
0,0,16558,99.98%
1,1,5145,31.07%


### phtn

In [112]:
fn = 'phtn'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(2, 23, 25, ['30502', '30500', '30390'])

In [117]:
df['phtn_b4idx'] = np.where(df.diag.isin(all_ls),1,0) 
patient(df, 'phtn_b4idx')

,phtn_b4idx,patid,rate
0,0,16558,99.98%
1,1,5145,31.07%


### BMI

In [119]:
fn = 'bmi'
ls_9 = pd.read_pickle(dx_path+fn+'_9.pickle')
ls_10 = pd.read_pickle(dx_path+fn+'_10.pickle')
all_ls = ls_9 + ls_10
len(ls_9), len(ls_10),len(all_ls), al_ls [:3]

(17, 23, 40, ['30502', '30500', '30390'])

In [120]:
df['bmi_b4idx'] = np.where(df.diag.isin(all_ls),1,0) 
patient(df, 'bmi_b4idx')

,bmi_b4idx,patid,rate
0,0,16562,100.0%
1,1,1805,10.9%


## Output label before index

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3869795 entries, 0 to 3869794
Data columns (total 18 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   patid                 int64 
 1   fst_dt                object
 2   icd_flag              int64 
 3   diag                  object
 4   index_day             object
 5   day_gap               int64 
 6   MI_b4Idx_oldCode      int64 
 7   alcohol_b4idx         int64 
 8   CAD_b4idx             int64 
 9   hypertension_b4idx    int64 
 10  diabeteMCIcode_b4idx  int64 
 11  MCI_sl_b4idx          int64 
 12  tobacco_b4idx         int64 
 13  PVD_b4idx             int64 
 14  dyslipidemia_b4idx    int64 
 15  stroke_b4idx          int64 
 16  phtn_b4idx            int64 
 17  bmi_b4idx             int64 
dtypes: int64(15), object(3)
memory usage: 531.4+ MB


In [123]:
df.to_csv(in_path+'temp_diagnosis_b4idx_label.csv',index=False)